In [ ]:
import glob
import subprocess

raw_files_path = '/Volumes/Seagate_storage/Foraging_study_copy2/'
exclude = ['101','115','206']
all_subj_dir = [x for x in glob.glob(raw_files_path + 'sub*') if not any(j in x for j in exclude)]
use_sb_epi = True  # Use scout file as initial high res?
prepro_file = 'preproc_scans_16Feb2022'

for subj_path in all_subj_dir:  # subj_path=all_subj_dir[5]
    subj_name = subj_path.split('/')[-1]
    
    print("\n--------Creating fsf for subject", subj_name)
    output_dir = subj_path + '/preproc_21Feb2022'
    save_fsf_folder = subj_path
    
    ########################## Edit preprocessing fsf #########################
    # Open template file
    prepro_template_path = '/Volumes/Seagate_storage/Foraging_study_copy2/fsf_templates/preprocess_template.fsf'
    with open(prepro_template_path) as file:
        prepro_template = file.readlines()
    
    # Customize paths for scan inputs
    path_to_replace = '/Volumes/MyPassport/Hailey_data_preproc/fMRI_data/preprocess/sub105/'
    new_path = raw_files_path + subj_name + '/'+prepro_file+'/'
    
    for line in range(len(prepro_template)):
        if path_to_replace in prepro_template[line]:
            prepro_template[line] = prepro_template[line].replace(path_to_replace, new_path)
            
    # Replace output directory
    outputdir_ind = [x for x in range(len(prepro_template)) if 'set fmri(outputdir)' in prepro_template[x]][0]
    prepro_template[outputdir_ind] = 'set fmri(outputdir) "'+output_dir+'"\n'
    
    # Get number of volumes from the EPI using fslval
    func = glob.glob(raw_files_path+subj_name+'/'+prepro_file+'/epiRun.nii.gz')[0]
    process = subprocess.Popen(['fslval', func, 'dim4'],
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    epi_vols, stderr = process.communicate()
    epi_vols = epi_vols.decode('utf-8').strip(' \n')
    nvol_ind = [x for x in range(len(prepro_template)) if 'set fmri(npts)' in prepro_template[x]][0]
    prepro_template[nvol_ind] = 'set fmri(npts) '+ epi_vols + '\n'
    
    # Get number of voxels from the EPI using fslval
    process = subprocess.Popen(['fslstats', func, '-v'],
                          stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    epi_vox, stderr = process.communicate()
    epi_vox = epi_vox.decode('utf-8').split(' ')[0]
    totalvox_ind = [x for x in range(len(prepro_template)) if 'set fmri(totalVoxels)' in prepro_template[x]][0]
    prepro_template[totalvox_ind] = 'set fmri(totalVoxels) '+epi_vox+'\n'

    # Write to first level file
    with open(save_fsf_folder+'/prepro_v2_21Feb2022.fsf', 'w') as f:
        for item in prepro_template:
            f.write("%s" % item)
            
    del prepro_template